In [100]:
import os
import numpy as np
import itertools
import math, random
random.seed = 42
import numpy as np
import open3d as o3d
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from path import Path
import scipy.spatial.distance
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

## Data preparation

In [602]:
path = Path('/Users/wuhongyu/code/Dataset/ModelNet40')

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
# classes

In [558]:
def read_off(file):
    off_header = file.readline().strip()
    if 'OFF' == off_header:
        n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    else:
        n_verts, n_faces, __ = tuple([int(s) for s in off_header[3:].split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces


def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))

    
    fig = go.Figure(data=data,
        layout=go.Layout(
            updatemenus=[dict(type='buttons',
                showactive=False,
                y=1,
                x=0.8,
                xanchor='left',
                yanchor='bottom',
                pad=dict(t=45, r=10),
                buttons=[dict(label='Play',
                    method='animate',
                    args=[None, dict(frame=dict(duration=50, redraw=True),
                        transition=dict(duration=0),
                        fromcurrent=True,
                        mode='immediate'
                        )]
                    )
                ])]
        ),
        frames=frames
    )

    return fig



def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

def pc_show_multi(object_coords,control_point_coords):
    """
    xs,ys,zs the coordinates from ML40 object;
    xp,yp,zp  the coordinated from FFD control points 
    """
    xs,ys,zs = object_coords
    xp,yp,zp = control_point_coords


    
    data = [go.Scatter3d(x=xs, y=ys, z=zs,mode='markers'),go.Scatter3d(x=xp, y=yp, z=zp,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

    

In [563]:
with open(path/"airplane/train/airplane_0001.off", 'r') as f:
    verts, faces = read_off(f)
    
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T
coords = np.column_stack((x,y,z))

len(coords), coords.shape

(90714, (90714, 3))

## Subsampling  and normalization

In [565]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
    
    
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud


In [566]:

with open(path/"airplane/train/airplane_0001.off", 'r') as f:
    verts, faces = read_off(f)
    
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T
coords = np.column_stack((x,y,z))



pointcloud = PointSampler(3000)((verts, faces))
norm_pointcloud = Normalize()(pointcloud)



## Visualization

In [567]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))

    
    fig = go.Figure(data=data,
        layout=go.Layout(
            updatemenus=[dict(type='buttons',
                showactive=False,
                y=1,
                x=0.8,
                xanchor='left',
                yanchor='bottom',
                pad=dict(t=45, r=10),
                buttons=[dict(label='Play',
                    method='animate',
                    args=[None, dict(frame=dict(duration=50, redraw=True),
                        transition=dict(duration=0),
                        fromcurrent=True,
                        mode='immediate'
                        )]
                    )
                ])]
        ),
        frames=frames
    )

    return fig



def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

def pc_show_multi(object_coords,control_point_coords):
    """
    xs,ys,zs the coordinates from ML40 object;
    xp,yp,zp  the coordinated from FFD control points 
    """
    xs,ys,zs = object_coords
    xp,yp,zp = control_point_coords


    
    data = [go.Scatter3d(x=xs, y=ys, z=zs,mode='markers'),go.Scatter3d(x=xp, y=yp, z=zp,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

    

In [595]:

with open(path/"piano/train/piano_0001.off", 'r') as f:
    verts, faces = read_off(f)
    
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T
coords = np.column_stack((x,y,z))



pointcloud = PointSampler(3000)((verts, faces))
norm_pointcloud = Normalize()(pointcloud)

pcshow(*norm_pointcloud.T)


# FFD

In [596]:
%matplotlib inline
import numpy as np
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

import pygem
print(pygem.__version__)
from pygem import FFD

2.0.0


In [597]:
def random_move(ffd):
    ffd = ffd 
    point =np.random.randint(0,3,size=[3])
    ffd.array_mu_x[point[0],point[1],point[2]] = np.random.uniform(0.5,1.5)
    ffd.array_mu_z[point[0],point[1],point[2]] = np.random.uniform(0.5,1.5)
    ffd.array_mu_z[point[0],point[1],point[2]] = np.random.uniform(0.5,1.5)

    return ffd 

ffd  =  FFD ([ 3 ,  3 ,  3 ])
# ffd.array_mu_x[1, 1, 1] = 1.5
# ffd.array_mu_y[1, 1, 1] = 1.5
# ffd.array_mu_z[1, 1, 1] = 1.5

# ffd.array_mu_x[2, 2, 2] = 0.5
# ffd.array_mu_y[2, 2, 2] = 0.5
# # ffd.array_mu_z[2, 2, 2] = 0.5

ffd.box_length = [2, 2, 2]


for i in range(6):
    ffd =  random_move(ffd)



# print(ffd)

## Translate the point clouds 

In [598]:
def make_open3d_point_cloud(xyz, color=None):
  #convert coordinates array to open 3d object
  pcd = o3d.geometry.PointCloud() 
  pcd.points = o3d.utility.Vector3dVector(xyz)
  if color is not None:
    pcd.colors = o3d.utility.Vector3dVector(color)
  return pcd

pcd = make_open3d_point_cloud(norm_pointcloud)

# calculate the mid point among control points
tx = ffd.control_points()[:,0].mean()
ty = ffd.control_points()[:,1].mean()
tz = ffd.control_points()[:,2].mean()

# move 
pcd.translate((tx,ty,tz),relative=False)



PointCloud with 3000 points.

##  FFD Transformation

In [599]:
new_mesh = ffd(np.asarray(pcd.points))
print(type(new_mesh), new_mesh.shape)

<class 'numpy.ndarray'> (3000, 3)


## Visualization

In [600]:
pc_show_multi(new_mesh.T,ffd.control_points().T)


In [601]:
pc_show_multi(new_mesh.T,np.asarray(pcd.points).T)
